## data generate

In [5]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from gensim.models import Word2Vec,KeyedVectors
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# 打开文件
text_path = '../data/Corpus.txt'
name_path = '../data/powerset.txt'
with open(text_path,'r') as f:
    texts = f.readlines()
f.close()
with open(name_path,'r') as g:
    names = g.readlines()
g.close()

In [7]:
# 统计标签组
'''label_set = []
for name in names:
    l = eval(name.split('\t',2)[-1])
    label_set.extend(l)
label_set = list(set(label_set))
print(len(label_set))
label2id = {}
for i,label in enumerate(label_set):
    label2id[label] = i
print(label_set)
print(label2id)'''
label2id = {}
with open('config/labels.txt','r') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip().split('\t')
    label2id[line[0]] = eval(line[1])
label_set = []
for key in label2id:
    label_set.append(key)
print(len(label_set))
print(label_set)
print(label2id)

34
['深圳1', '市委直属', '教育', '市人大直属', '罗湖', '市政协直属', '军检法机构1', '一般院校', '宝安', '理工方向1', '经济金融', '公安', '街道基层', '卫生环保', '基层工作', '双一流大学', '龙华', '军检法机构', '龙岗', '福田', '交通运输', '盐田', '光明', '港澳外事', '大鹏新区', '深圳市外1', '科技技术', '党务工作', '深圳', '市政府直属', '国企事业单位', '坪山', '法务纪检', '南山']
{'深圳1': 0, '市委直属': 1, '教育': 2, '市人大直属': 3, '罗湖': 4, '市政协直属': 5, '军检法机构1': 6, '一般院校': 7, '宝安': 8, '理工方向1': 9, '经济金融': 10, '公安': 11, '街道基层': 12, '卫生环保': 13, '基层工作': 14, '双一流大学': 15, '龙华': 16, '军检法机构': 17, '龙岗': 18, '福田': 19, '交通运输': 20, '盐田': 21, '光明': 22, '港澳外事': 23, '大鹏新区': 24, '深圳市外1': 25, '科技技术': 26, '党务工作': 27, '深圳': 28, '市政府直属': 29, '国企事业单位': 30, '坪山': 31, '法务纪检': 32, '南山': 33}


In [8]:
x_train = []
x_test = []
y_train = []
y_test = []
for i in range(len(texts)):
    _,flag,labels = names[i].split('\t',2)
    labels = eval(labels)
    labels = [label2id[x] for x in labels]
    y = np.zeros(len(label_set))
    for ind in labels:
        y[ind] = 1
    if flag == 'train':
        x_train.append(texts[i].strip())
        y_train.append(y)
    elif flag == 'test':
        x_test.append(texts[i].strip())
        y_test.append(y)
y_train = np.array(y_train)
y_test = np.array(y_test)
#print(len(x_train),x_train)
print('*********')
#print(len(x_test),x_test)
print('*********')
print(y_train.shape,y_train)
print('*********')
print(y_test.shape,y_test)

*********
*********
(11481, 34) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
*********
(5741, 34) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [147]:
'''x = x_train
print(len(x))
print(x[0:2])
'''

'x = x_train\nprint(len(x))\nprint(x[0:2])\n'

## W2V

In [148]:
Model_path = 'model/w2vModel3.txt'
Mymodel = KeyedVectors.load_word2vec_format(Model_path, binary=False)
#print(Mymodel['鄂南'])
def doc_emb(s):
    s = s.split()
    result = 0
    for item in s:
        result += Mymodel[item]
    return result/len(s)

In [149]:
for i,s in enumerate(x_train):
    x_train[i] = doc_emb(s)
x_train = np.array(x_train)
for i,s in enumerate(x_test):
    x_test[i] = doc_emb(s)
x_test = np.array(x_test)
print(x_train.shape,x_train[0:1])
print('*********')
print(x_test.shape,x_test[0:1])

(11481, 5) [[-1.9764199   0.6711333   2.6666346  -0.32828453  0.666273  ]]
*********
(5741, 5) [[-1.9183024   0.62873656  2.7205193  -0.38334066  0.7298054 ]]


In [150]:
model = OneVsRestClassifier(svm.SVC(kernel='linear'))
clf = model.fit(x_train, y_train)

In [151]:
def evaluate(x,y):
    c = (x + y)[0]
    m,n = 0,0
    #print(c)
    for num in c:
        #print(num)
        if num > 0.0:
            m += 1
        if num == 2.0:
            n += 1
    return n/m

In [152]:
score = 0
for i in range(x_test.shape[0]):
    y_predict = clf.predic           t(np.array([x_test[i]]))
    score += evaluate(y_predict,y_test[i])
score = score/x_test.shape[0]
print(score)

0.7943679962840371


In [142]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

## KGE

In [9]:
Model_path = 'model/kge1.txt'
f = open(Model_path,'rb')
Mymodel = pickle.load(f)
f.close()
#print(Mymodel['鄂南'])
def doc_emb(s):
    s = s.split()
    result = 0
    for item in s:
        result += Mymodel[item].detach().numpy()
    return result/len(s)

In [10]:
for i,s in enumerate(x_train):
    x_train[i] = doc_emb(s)
x_train = np.array(x_train)
for i,s in enumerate(x_test):
    x_test[i] = doc_emb(s)
x_test = np.array(x_test)
print(x_train.shape,x_train[0:1])
print('*********')
print(x_test.shape,x_test[0:1])

(11481, 300) [[-0.0862608  -0.05608232 -0.13693452  0.1446248  -0.07380942  0.08100955
  -0.0710175   0.19466478  0.02078468  0.02060594 -0.13324702 -0.15828931
   0.06589845 -0.13175388 -0.15458873 -0.17275885 -0.19655243  0.03675377
  -0.06032414 -0.12580003  0.20336418 -0.31082505 -0.09533659  0.19135836
   0.05053294 -0.34484673 -0.04444493 -0.03692457  0.18333444 -0.16159938
  -0.2788506  -0.01312388  0.10778818  0.01640877 -0.05742651 -0.07790904
   0.08265409 -0.10446168  0.2507329   0.01917612 -0.08266401  0.07949202
   0.19795117  0.0072046  -0.22024718  0.02946759  0.00650709  0.13854086
   0.03132294  0.23981547  0.00574612  0.07103635 -0.00628256  0.03451007
  -0.06209029 -0.07955387 -0.19795251  0.01136472 -0.11751206  0.20898965
   0.02256358 -0.09646541 -0.29961485  0.32781976 -0.00779665 -0.02768272
   0.02342199 -0.03432281 -0.06653438 -0.03905541 -0.26659706  0.06874431
  -0.15845735  0.04579316 -0.02242407 -0.07156527 -0.26021427  0.19883889
  -0.24606869 -0.1150887 

In [11]:
model = OneVsRestClassifier(svm.SVC(kernel='linear',probability=True))
clf = model.fit(x_train, y_train)

In [12]:
def evaluate(x,y):
    c = (x + y)[0]
    m,n = 0,0
    #print(c)
    for num in c:
        #print(num)
        if num > 0.0:
            m += 1
        if num == 2.0:
            n += 1
    return n/m

In [13]:
score = 0
for i in range(x_test.shape[0]):
    y_predict = clf.predict(np.array([x_test[i]]))
    score += evaluate(y_predict,y_test[i])
score = score/x_test.shape[0]
print(score)

0.8046710793706081


In [16]:
f = open('model/kg_svc_1.model','wb')
pickle.dump(clf,f)
f.close()

In [18]:
f = open('model/kg_svc_1.model','rb')
liu = pickle.load(f)
f.close()

## demo

In [6]:
digits = load_digits()

x, y = digits.data, digits.target
y = label_binarize(y, classes=list(range(10)))
print(x.shape,x)
print('*********')
print(y.shape,y)
print('*********')
x_train, x_test, y_train, y_test = train_test_split(x, y)
print(x_train.shape,x_train)
print('*********')
print(x_test.shape,x_test)
print('*********')
print(y_train.shape,y_train)
print('*********')
print(y_test.shape,y_test)

(1797, 64) [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
*********
(1797, 10) [[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 0]]
*********
(1347, 64) [[ 0.  0.  1. ...  0.  0.  0.]
 [ 0.  0.  0. ... 14.  0.  0.]
 [ 0.  2. 13. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 15.  4.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  1.  7. ...  6.  0.  0.]]
*********
(450, 64) [[ 0.  0. 15. ...  0.  0.  0.]
 [ 0.  0.  5. ... 13. 12.  4.]
 [ 0.  0. 11. ... 13. 16.  7.]
 ...
 [ 0.  0.  0. ... 15.  0.  0.]
 [ 0.  0.  1. ...  8.  0.  0.]
 [ 0.  0.  0. ...  8.  0.  0.]]
*********
(1347, 10) [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
*********
(450, 10) [[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 

In [34]:
#model = OneVsRestClassifier(svm.SVC(kernel='linear'))
model = OneVsRestClassifier(SGDClassifier(loss='log'))
clf = model.fit(x_train, y_train)
print(type(model))
print(type(clf))
print(model is clf)
print(id(model) is id(clf))
#print(clf.multilabel_)

<class 'sklearn.multiclass.OneVsRestClassifier'>
<class 'sklearn.multiclass.OneVsRestClassifier'>
True
False


In [35]:
clf = model.partial_fit(x_test, y_test)

In [36]:
print(clf.score(x_train, y_train))
print(clf.score(x_test, y_test))
print(np.argmax(y_test, axis=1))
print(np.argmax(clf.decision_function(x_test), axis=1))

0.8559762435040832
0.8311111111111111
[5 2 2 7 3 6 8 5 1 5 0 1 9 3 8 6 4 8 9 2 2 0 7 1 1 1 5 2 0 4 8 2 1 7 9 6 5
 6 8 0 9 6 1 4 7 1 3 4 4 4 3 1 8 5 1 5 8 1 2 9 0 3 9 1 7 5 9 1 8 2 8 5 4 4
 2 0 0 9 6 3 1 0 5 4 0 0 6 6 8 9 1 6 4 5 2 3 6 5 5 1 4 7 4 1 8 2 5 6 8 0 5
 9 1 4 3 9 3 1 3 2 2 2 9 6 1 2 8 0 2 3 2 5 9 4 6 4 5 0 5 0 0 6 5 2 1 3 0 2
 0 5 8 1 0 3 0 5 8 2 8 2 5 9 6 3 0 8 0 9 6 3 6 9 4 3 1 8 1 7 5 3 6 1 1 0 5
 7 2 4 0 0 4 5 7 7 6 3 2 7 0 7 1 4 0 7 7 2 0 3 2 2 8 5 6 2 7 8 0 8 9 0 9 2
 8 1 3 6 3 7 5 8 3 7 4 1 2 0 4 6 4 2 7 6 2 5 4 7 6 7 3 7 9 3 7 4 9 4 3 7 5
 5 4 9 9 0 7 4 6 4 8 4 8 2 3 9 9 7 7 4 0 9 3 2 7 9 8 7 5 1 5 3 6 2 5 8 7 7
 8 1 2 2 0 1 4 7 3 4 2 9 4 6 5 2 4 6 4 8 4 2 0 5 3 0 0 0 7 1 4 5 8 2 0 9 9
 1 0 7 2 2 9 0 7 7 8 7 7 2 8 8 6 0 1 1 5 4 2 6 3 0 3 3 7 1 9 1 8 4 8 6 3 7
 5 9 3 8 5 4 1 1 5 1 6 5 0 7 3 4 6 2 9 2 4 5 7 2 6 0 3 8 9 4 6 1 6 9 1 5 5
 1 6 0 2 1 6 1 7 7 1 3 5 7 8 6 3 5 2 2 2 3 1 2 5 6 2 0 0 7 3 2 6 8 1 6 2 0
 4 1 8 1 6 1]
[5 2 2 7 3 6 8 5 1 5 0 1 7 3 8 6 4 8 9 2 2 0 7 1

In [21]:
clf.predict(x_test[0:10])

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])